# Convolutional Neural Networks - Smile Classifier

In this notebook, I implement a CNN to classify face images based on smiles. I also explore data augmentation techiniques to enhance the model's performance. Let's jump in right away and load the celebA dataset.

# Loading the CelebA dataset

CelebFaces Attributes Dataset, or CelebA for short, is an image dataset that identifies celebrity face attributes. It contains 202,599 face images across five landmark locations, with 40 binary attribute annotations for each image. 

Tought the dataset is available through the PyTorch's `torchvision` module, the link appears to be unstable. So, I downloaded the dataset manually using this [link](https://drive.google.com/file/d/1m8-EBPgi5MRubrm6iQjafK2QMHDBMSfJ/view?usp=sharing)

In [7]:
import torch
import torchvision
from torchvision import transforms

image_path = './dataset'

#Load training partition of the dataset
celeba_train_dataset = torchvision.datasets.CelebA(
    root=image_path, split='train',
    target_type='attr', download=False
)

#Load validation partition of the dataset
celeba_valid_dataset = torchvision.datasets.CelebA(
    root=image_path, split='valid',
    target_type='attr', download=False
)

#Load testing partition of the dataset
celeba_valid_dataset = torchvision.datasets.CelebA(
    root=image_path, split='test',
    target_type='attr', download=False
)

# Data augmentation

**Data augmentation** refers to a set of techniques for dealing with cases where the training data is limited. Those techniques let us modify or even synthesize more data to bring more variation in the dataset which is good.

To augment our dataset, we need to perform "transformations" on it. Remember, in the folder 03 in `mpl-torch.ipynb`, I said the following:

> I import the torchvision and **transforms** modules. The second module[transform], as the name suggests, let us perform common transformations on **image** data. According to the documentation, Transforms are common image transformations available in the torchvision.transforms module.
>
>
> Another interesting feature is that transform operations can be **chained** together using `Compose`.

Here again, I will use the `transform` module to perform the transformations and use `Compose` to chain those transformations. Let's do this!

In [19]:
celeba_train_dataset.attr.shape

torch.Size([162770, 40])